In [1]:
%pip install llama-index-llms-openai
!pip install llama-index
!pip install --upgrade llama-index-core llama-index-llms-openai llama-index-utils-workflow


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23
  Attempting uninstall: llama-index-llms-openai
    Found existing installation: llama-index-llms-openai 0.2.16
    Uninstalling llama-index-llms-openai-0.2.16:
      Successfully uninstalled llama-index-llms-openai-0.2.16


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.23 requires llama-index-core<0.12.0,>=0.11.23, but you have llama-index-core 0.12.5 which is incompatible.
llama-index 0.11.23 requires llama-index-llms-openai<0.3.0,>=0.2.10, but you have llama-index-llms-openai 0.3.10 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.5 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-llms-openai<0.3.0,>=0.2.9, but you have llama-index-llms-openai 0.3.10 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.5 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-llms-openai<0.3.0,>=0.2.0, but you have llama-index-llms-openai 0.3.10 which is incompatible.
llama-index-embeddings-

In [2]:

from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
import random


In [3]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:

In [4]:
class OllamaGenerator(Workflow):
    @step
    async def generate(self, ev: StartEvent) -> StopEvent:
        response = await llm.acomplete(ev.query)  # Query Ollama asynchronously
        return StopEvent(result=str(response))


In [5]:
w = OllamaGenerator(timeout=10, verbose=False)
result = await w.run(query="What's LlamaIndex?")
print(result)


DEBUG:llama_index.core.instrumentation.dispatcher:Failed to reset active_span_id: <Token var=<ContextVar name='active_span_id' default=None at 0x000001E3C0E8A660> at 0x000001E3A9141B00> was created in a different Context
Failed to reset active_span_id: <Token var=<ContextVar name='active_span_id' default=None at 0x000001E3C0E8A660> at 0x000001E3A9141B00> was created in a different Context


WorkflowRuntimeError: Error in step 'generate': name 'llm' is not defined

In [7]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
from llama_index.core.workflow import draw_all_possible_flows
from llama_index.llms.ollama import Ollama  # Ollama integration
import random

# Define the Ollama instance globally or as part of the workflow
class OllamaGenerator(Workflow):
    def __init__(self, timeout: int, verbose: bool = False):
        super().__init__(timeout=timeout, verbose=verbose)
        self.llm = Ollama(
            base_url="http://localhost:11434",  # Your local Ollama setup
            model="llama3.2:latest",            # The preferred model
            temperature=0.1                     # Controlled temperature for deterministic results
        )

    @step
    async def generate(self, ev: StartEvent) -> StopEvent:
        response = await self.llm.acomplete(ev.query)  # Use the llm defined in the class
        return StopEvent(result=str(response))

# Initialize and run the workflow
w = OllamaGenerator(timeout=10, verbose=False)
result = await w.run(query="What's cosine similarity?")
print(result)


DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Users\\Muthu\\.conda\\envs\\llamaindex\\Library\\ssl\\cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
connect_tcp.started host='localhost' port=11434 local_address=None timeout=30.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001E3AA391190>
connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001E3AA391190>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete


In [8]:
draw_all_possible_flows(OllamaGenerator, filename="trivial_workflow.html")


C:\Users\Muthu\AppData\Local\Temp\ipykernel_29744\1234723504.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(OllamaGenerator, filename="trivial_workflow.html")


trivial_workflow.html


In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

# vector_index = VectorStoreIndex.from_documents(documents, embed_model=ollama_embedding)
vector_index = VectorStoreIndex.from_documents(documents)


In [7]:
qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Please write the answer in the style of {tone_name}
Query: {query_str}
Answer: \
"""

In [12]:
# Create a PromptTemplate object
prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

# Apply partial formatting to set the tone_name
partial_prompt_tmpl = prompt_tmpl.partial_format(tone_name="Shakespeare")

In [17]:
formatted_prompt = partial_prompt_tmpl.format(
    context_str="In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters",
    query_str="How many params does Llama 2 have?"
)

In [ ]:
response = vector_index.as_query_engine(
    llm=ollama_llm
).query(formatted_prompt)


In [ ]:
print("\nResponse from the Query Engine:")
print(response)